# Cleaning the Data and creating a sparce matrix

I start by importing the basic libraries and the data from a file. 

In [1]:
import pandas as pd
import numpy as np

### This is what the data looks like:

In [2]:
legislatorsData = pd.read_csv("../data/legislators.csv")
legislatorsData.head()
legislatorsData.columns
legislators = pd.DataFrame(legislatorsData)
legislators.head()

,title,firstname,middlename,lastname,name_suffix,nickname,party,state,district,in_office,...,govtrack_id,crp_id,twitter_id,congresspedia_url,youtube_url,facebook_id,official_rss,senate_class,birthdate,oc_email
0,Rep,Neil,NaN,Abercrombie,NaN,NaN,D,HI,1,0,...,400001,N00007665,neilabercrombie,http://www.opencongress.org/wiki/Neil_Abercrombie,http://youtube.com/hawaiirep1,NaN,NaN,NaN,1938-06-26,NaN
1,Rep,Gary,L.,Ackerman,NaN,NaN,D,NY,5,0,...,400003,N00001143,repgaryackerman,http://www.opencongress.org/wiki/Gary_Ackerman,http://youtube.com/RepAckerman,RepAcherman,NaN,NaN,1942-11-19,NaN
2,Rep,Robert,B.,Aderholt,NaN,NaN,R,AL,4,1,...,400004,N00003028,Robert_Aderholt,http://www.opencongress.org/wiki/Robert_Aderholt,http://youtube.com/RobertAderholt,19787529402,NaN,NaN,1965-07-22,Rep.Aderholt@opencongress.org
3,Sen,Daniel,Kahikina,Akaka,NaN,NaN,D,HI,Junior Seat,0,...,300001,N00007653,NaN,http://www.opencongress.org/wiki/Daniel_Akaka,http://youtube.com/senatorakaka,danielakaka,NaN,I,1924-09-11,NaN
4,Sen,Wayne,A.,Allard,NaN,NaN,R,CO,Senior Seat,0,...,300003,N00009082,NaN,http://www.opencongress.org/wiki/Wayne_Allard,NaN,NaN,NaN,II,1943-12-02,NaN


In [3]:
from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize

In [4]:
def requestWords( id ):
    id = str(id)
    url = "http://capitolwords.org/api/1/phrases.json?entity_type=legislator&entity_value="+id+"&apikey=0bf8e7eb6ce146f48217bfee767c998d"
    request=Request(url)
    response = urlopen(request)
    contents = response.read()
    len(contents)
    if len(contents) > 2:
        data = json.loads(contents)
        words = json_normalize(data)
        list_of_words = words.ngram.tolist()
        string_of_words ="|".join(list_of_words)
        return string_of_words
    else:
        return np.nan

In [5]:
legislators['favorite_words'] = legislators.apply(lambda row: requestWords(row['bioguide_id']),axis=1)

In [6]:
print legislators.favorite_words.head(3)
print "All entries before getting rid of entris with no words:", len(legislators.favorite_words)

0    hawaiian|hawaii|hawaiians|hawaii's|kalaupapa|e...
1    queens|rabbi|jewish|bayside|flushing|nassau|br...
2    aderholt|requesting|irons|huntsville|alabama|r...
Name: favorite_words, dtype: object
All entries before getting rid of entris with no words: 897


In [7]:
legislators_words = legislators[legislators.favorite_words.notnull()]

In [ ]:
print "Number of legislators with word record:", len(legislators_words.favorite_words)

Number of legislators with word record: 763


In [ ]:
favorite_words = legislators_words.favorite_words.str.get_dummies(sep = "|")
print favorite_words.head(3)
favorite_words.columns[:100]

In [ ]:
favorite_words.shape

In [ ]:
favorite_words.columns[260:300]

In [ ]:
favorite_words.columns[760:800]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
vectorizer

#### Corpus
I used the whole must repited words to have more of a global tf-idf

In [ ]:
corpus = favorite_words.columns.tolist()
corpus[:3]

In [ ]:
X = vectorizer.fit_transform(corpus)

In [ ]:
analyze = vectorizer.build_analyzer()
print analyze("economy a this")
vectorizer.get_feature_names()[910:920]

In [ ]:
vectorizer.vocabulary_.get('document') #not seen in the training corpus will be completely ignored in future calls to the transform method

In [ ]:
vectorizer.transform(['Something completely unrelated']).toarray()

### Biagram
not sure if i shoud add Biagrams since the corpus is made up single words and a lot of of number dropping politicians love to do.

### Transformer

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
transformer = TfidfTransformer()
transformer

In [ ]:
tfidf = transformer.fit_transform(favorite_words)
tfidf_array = tfidf.toarray()
tfidf_array.shape
tfidf_array[20].max()
transformer.idf_

### Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1)
vectorizer.fit_transform(corpus)
vec_idf = vectorizer.idf_
print len(vec_idf)

In [ ]:
words_weight = pd.DataFrame(tfidf_array, index=legislators_words.index , columns=corpus)
print legislators_words.index
print words_weight.index

In [ ]:
capitol_words = legislators_words.merge(words_weight, right_index=True, left_index=True)

In [ ]:
capitol_words.head()

In [ ]:
del capitol_words["a"]
column_names_capitol = capitol_words.columns.tolist()
word_column_names = column_names_capitol[806:]
number_column_names = column_names_capitol[30:805]

In [ ]:
capitol_words[word_column_names].head()

In [ ]:
capitol_words[word_column_names].sum().max()

### Now I will make another DataFrame removing the words with the max counts


In [ ]:
total_word_counts = capitol_words[word_column_names].sum()
max_sum_col = total_word_counts[total_word_counts==total_word_counts.max()]
max_sum_col.index[0]

In [ ]:
print "Words before cleanse for maximizing variance:", len(word_column_names)

## Getting rid of the words that 95% of the people said
Because they don't add much in to analyzing what makes legislators different from one another.

In [ ]:
def percentage_global_mention(word_column_names):
    for word in word_column_names:
        not_mentioned_mask = capitol_words[word_column_names][word]==0.0
        not_mentioned_count = capitol_words[word_column_names][word].mask(not_mentioned_mask).count()
        index_count = capitol_words.count()[0]
        percentage = float(not_mentioned_count)/index_count
        if percentage > 0.95:
            print percentage , word
            del capitol_words[word]
        else:
            print "Nothing to worry about"

### Or a better way to do it:
credits to Sergey

In [ ]:

word_frequencies = (capitol_words[word_column_names]>0).astype(int).sum(axis=0).astype(float)/capitol_words.shape[0]
most_frequent_words = word_frequencies[word_frequencies>.95].index
most_frequent_words

In [ ]:
word_frequencies = (capitol_words[word_column_names]>0).astype(int).sum(axis=0)
word_frequencies.max()

In [ ]:
capitol_words.party_x.unique()
party_mask = capitol_words.party_x!="I"
two_party_words = capitol_words[party_mask]
print "Entries before getting rid of independents:", capitol_words.shape[0]
print "Entries after getting rid of independents:", two_party_words.shape[0]
print "Number of independents:", (capitol_words.shape[0])-(two_party_words.shape[0])

In [ ]:
two_party_words.to_csv(path_or_buf="../data/two.csv")